In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set()

In [2]:
main_df = pd.read_pickle('wnba_total_df.pkl')

In [3]:
main_df.dropna(subset = ['PER', 'TS%', 'TOV%'], inplace=True)

In [4]:
main_df.reindex()

,Player,Year,Team,Position,Games_Played,Games_Started,Min_Played,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/40,Start_Over_Played,Points_Per_Game,Min_Played_Per_Game,DRB%
0,Tajama Abraham,1997,SAC,C,28,5,422,48,126,0.381,...,25.6,20.3,-0.6,-0.2,-0.8,-0.077,0.178571,4.357143,15.071429,0.8
1,Marcie Alberts,1997,CLE,G,5,0,30,0,2,0.000,...,60.0,7.8,-0.1,0.0,-0.1,-0.138,0.000000,0.000000,6.000000,2.1
2,Monique Ambers,1997,PHO,F,19,0,85,4,9,0.444,...,31.0,12.0,0.0,0.1,0.1,0.038,0.000000,0.736842,4.473684,0.7
3,Yvette Angel,1997,SAC,G,5,3,90,7,16,0.438,...,31.6,12.6,-0.1,0.0,-0.1,-0.061,0.600000,2.800000,18.000000,3.6
4,Janeth Arcain,1997,HOU,G,28,27,784,110,250,0.440,...,18.9,21.2,2.3,1.0,3.3,0.169,0.964286,10.892857,28.000000,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725,Han Xu,2019,NYL,C,18,0,143,24,58,0.414,...,3.3,18.6,0.0,0.0,0.0,-0.006,0.000000,3.000000,7.944444,4.4
3726,Jackie Young,2019,LVA,G,34,34,767,76,236,0.322,...,16.5,18.0,0.1,1.2,1.3,0.067,1.000000,6.617647,22.558824,4.5
3727,Tamera Young,2019,LVA,F,34,4,628,68,179,0.380,...,18.9,17.4,-0.4,1.0,0.6,0.038,0.117647,5.294118,18.470588,4.6
3728,Amanda Zahui B.,2019,NYL,C,24,23,558,81,173,0.468,...,14.8,17.0,0.6,0.8,1.4,0.098,0.958333,8.625000,23.250000,10.2


In [5]:
reduced_df = main_df.loc[:, ['Min_Played_Per_Game', 'Start_Over_Played', 'Points_Per_Game', 'PER',
                             'TS%','ORB%','DRB%','AST%','STL%','BLK%','TOV%','USG%', 'WS/40']]

In [143]:
sns.pairplot(reduced_df)
plt.savefig('pairplot.png')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <function flush_figures at 0x7ffe1cb41170> (for post_execute):



KeyboardInterrupt



In [6]:
X_base = reduced_df[['Start_Over_Played', 'Points_Per_Game', 'PER','TS%','ORB%','DRB%','AST%','STL%','BLK%','TOV%','USG%', 'WS/40']]
y_base = reduced_df['Min_Played_Per_Game']
X, y = np.array(X_base), np.array(y_base)

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X , y, test_size=0.20, random_state=69)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=420)

In [8]:
list(zip(X_base, linear_model.coef_))

NameError: name 'linear_model' is not defined

In [134]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s = []

In [135]:
for train_ind, val_ind in kf.split(X,y):
    X_train_val, y_train_val = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
print('Simple regression scores: ', cv_lm_r2s)
print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')

Simple regression scores:  [0.9080723755229876, 0.9148732987666309, 0.919616956914352, 0.9224420628847748, 0.9132487224257125]
Simple mean cv r^2: 0.916 +- 0.005


In [136]:
list(zip(X_base, linear_model.coef_))

[('Start_Over_Played', 8.410084944576498),
 ('Points_Per_Game', 1.3543837372475174),
 ('PER', -0.016508839666782762),
 ('TS%', 4.986812226084897),
 ('ORB%', 0.07998024826230744),
 ('DRB%', -0.02557616559836022),
 ('AST%', 0.10293493315377746),
 ('STL%', 0.3210358872033323),
 ('BLK%', -0.03954729319500921),
 ('TOV%', -0.03566949842590715),
 ('USG%', -0.41762768586790466),
 ('WS/40', -8.003960675972433)]